In [2]:
import train
import torch
model_type = 'xlstm'
model = train.new_model(model_type)
model.to('cpu')
train.train(model, model_type)

c:\Users\min\anaconda3\Lib\site-packages\torch\nn\init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


RuntimeError: mixed dtype (CPU): all inputs must share same datatype.

In [2]:
import train
import torch
import processing
import configs.common as cc
model_type = 'xlstm'
model = train.new_model(model_type)
train_dataloader, test_dataloader = processing.get_train_test_dataloaders('..\\dataset\\np_dataset')
metadata_vocab_size = processing.get_metadata_vocab_size()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=cc.config.values.learning_rate)

# Training loop
num_epochs = cc.config.values.epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_idx, (src, trg, metadata) in enumerate(train_dataloader):
        output = model(src.float())
        output = output.reshape(-1, model.vocab_size)  # Flatten the output to [batch_size * seq_len, vocab_size]
        trg = trg.view(-1)  # Flatten the target to [batch_size * seq_len]

        loss = criterion(output, trg)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():
        for src, trg, metadata in test_dataloader:
            src, trg = src, trg
            output = model(src)
            output = output.reshape(-1, model.vocab_size)
            trg = trg.view(-1)
            val_loss += criterion(output, trg).item()
    
    avg_val_loss = val_loss / len(test_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

print("Training complete!")

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [10]:
import torch.nn as nn
import torch.nn.functional as F
from models.xlstm.utils import BlockDiagonal, CausalConv1D
x = torch.randn(8, 128, 128) 
ln = nn.LayerNorm(128)
x = ln(x.float())

conv = CausalConv1D(128, 128, int(128/8))
drop = nn.Dropout(0.01)
F.silu( drop(conv( x.transpose(1, 2) ).transpose(1, 2) ) )

tensor([[[-4.9027e-02, -6.5153e-02, -7.2500e-02,  ...,  2.3825e-01,
          -6.5157e-03, -7.2104e-04],
         [-7.4015e-02,  8.5847e-03, -8.6271e-03,  ..., -4.3834e-02,
          -6.0125e-02,  1.3329e-01],
         [ 1.8839e-01,  1.6753e-01,  1.2251e-01,  ..., -3.4827e-02,
          -6.5553e-02, -5.6386e-02],
         ...,
         [ 9.5655e-01,  9.6815e-02, -2.6273e-01,  ...,  6.4457e-02,
           5.0679e-01, -2.0191e-01],
         [ 6.0297e-01, -2.0983e-01, -2.0662e-01,  ...,  1.1851e-01,
           4.4244e-01, -1.6025e-01],
         [-5.3179e-02,  8.7127e-02, -9.3752e-02,  ...,  5.9002e-01,
           3.9868e-01, -4.1369e-02]],

        [[ 5.6037e-03,  2.9015e-02, -5.0691e-02,  ...,  5.8675e-02,
           1.0559e-01,  4.9357e-02],
         [ 1.4903e-01,  3.4690e-02, -1.2088e-01,  ..., -5.1628e-02,
          -3.0116e-02, -1.3277e-01],
         [-1.2580e-01, -1.4506e-01,  9.8312e-02,  ..., -2.1266e-01,
          -2.0533e-01, -1.1790e-01],
         ...,
         [ 7.5295e-02, -2

In [8]:
x.shape

torch.Size([8, 128])

In [2]:
train.get_xlstm_dict()

namespace(block_size=252,
          n_embd=128,
          n_head=8,
          n_layer=8,
          feedforward_dim=2048,
          vocab_size=835,
          block_len=128,
          device='cpu',
          metadata_dims=namespace(composer=8),
          dropout=0.01,
          epochs=200,
          eval_interval=100,
          save_interval=500,
          learning_rate=0.1,
          eval_iters=200,
          test_ratio=0.2,
          batch_size=8)